In [ ]:
pip install python-chess

In [ ]:
import pandas as pd
import numpy as np
import chess.pgn
import chess
from random import randint
from random import shuffle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, losses
from keras import regularizers

In [ ]:
siamese_model = tf.keras.models.load_model("/kaggle/input/final2/siamese_250000.h5")

In [ ]:
def fen2bitstring(fen):
    board = chess.Board(fen)
    bitboard = np.zeros(64*6*2+5)

    piece_idx = {'p': 0, 'n': 1, 'b': 2, 'r': 3, 'q': 4, 'k': 5}

    for i in range(64):
        if board.piece_at(i):
            color = int(board.piece_at(i).color) + 1
            bitboard[(piece_idx[board.piece_at(i).symbol().lower()] + i * 6) * color] = 1

    bitboard[-1] = int(board.turn)
    bitboard[-2] = int(board.has_kingside_castling_rights(True))
    bitboard[-3] = int(board.has_kingside_castling_rights(False))
    bitboard[-4] = int(board.has_queenside_castling_rights(True))
    bitboard[-5] = int(board.has_queenside_castling_rights(False))

    return bitboard
    

In [ ]:
def randPosiGeneratorFromGame(game):
    random_positions_array = []
    positions = []
    board = game.board()

    for move in game.mainline_moves():
        if not board.is_capture(move):
            position = board.fen()
            bitstring_position = fen2bitstring(position)
            positions.append(bitstring_position)

        board.push(move)

        no_capture_posis = len(positions)
    indices = list(range(5, no_capture_posis))
    shuffle(indices)
    selected_indices = indices[:10]

    random_positions_array = [positions[index] for index in selected_indices]

    return random_positions_array

In [ ]:
pgn_file = open("../input/ccrl-dataset/CCRLdb.pgn")

random_positions_white_win_bitstring = []
random_positions_black_win_bitstring = []

i = 0
while i < (450000 + 125000):
    if i > 450000:
        game = chess.pgn.read_game(pgn_file)
        if game.headers["Result"] == "1-0":
            temp1 = randPosiGeneratorFromGame(game)
            random_positions_white_win_bitstring += temp1
        elif game.headers["Result"] == "0-1":
            temp2 = randPosiGeneratorFromGame(game)
            random_positions_black_win_bitstring += temp2
        if game is None:
            break
    i = i + 1
        
pgn_file.close()

In [ ]:
len(random_positions_white_win_bitstring)

In [ ]:
win = [1]
white_win_labels = []
for i in range(len(random_positions_white_win_bitstring)):
    white_win_labels.append(win)
loss = [0]
black_win_labels = []
for i in range(len(random_positions_black_win_bitstring)):
    black_win_labels.append(loss)

In [ ]:
white_win_bitstring_tensor = tf.convert_to_tensor(random_positions_white_win_bitstring, dtype=tf.float32)

white_win_labels_tensor = tf.convert_to_tensor(white_win_labels, dtype=tf.int32)

black_win_bitstring_tensor = tf.convert_to_tensor(random_positions_black_win_bitstring, dtype=tf.float32)

black_win_labels_tensor = tf.convert_to_tensor(black_win_labels, dtype=tf.int32)

In [ ]:
white_win_dataset = tf.data.Dataset.from_tensor_slices(white_win_bitstring_tensor)
black_win_dataset = tf.data.Dataset.from_tensor_slices(black_win_bitstring_tensor)

white_win_labels_dataset = tf.data.Dataset.from_tensor_slices(white_win_labels_tensor)
black_win_labels_dataset = tf.data.Dataset.from_tensor_slices(black_win_labels_tensor)

In [ ]:
paired_dataset = tf.data.Dataset.zip((white_win_dataset, black_win_dataset))
paired_labels_dataset = tf.data.Dataset.zip((white_win_labels_dataset, black_win_labels_dataset))

In [ ]:
dataset = tf.data.Dataset.zip((paired_dataset, paired_labels_dataset))

In [ ]:
trained_pos2vec_weights = np.load('/kaggle/input/pos2vec-final-draft1-200000/pos2vec_final_draft1_200000.npy', allow_pickle = True)
weights_encoder1 = trained_pos2vec_weights[0]
weights_encoder2 = trained_pos2vec_weights[1]
weights_encoder3 = trained_pos2vec_weights[2]
weights_encoder4 = trained_pos2vec_weights[3]

In [ ]:
# Weights are taken after training on pos2vec first
def create_pos2vec_model():
    dbn_model = tf.keras.Sequential([
    layers.Dense(600, activation='relu', input_shape=(773,)),
    layers.Dense(400, activation='relu', input_shape=(600,)),
    layers.Dense(200, activation='relu', input_shape=(400,)),
    layers.Dense(100, activation='relu', input_shape=(200,)),
    layers.Dense(100, activation='relu')
    ])
    # Set the weights for the DBN layers
    dbn_model.layers[0].set_weights(weights_encoder1)  # First encoder layer
    dbn_model.layers[1].set_weights(weights_encoder2)  # Second encoder layer
    dbn_model.layers[2].set_weights(weights_encoder3)  # Third encoder layer
    dbn_model.layers[3].set_weights(weights_encoder4)  # Fourth encoder layer
    #dbn_model.trainable = False

    return dbn_model

In [ ]:
def create_siamese_network():
    dbn = create_pos2vec_model()
    
    input1 = layers.Input(shape = (773,))
    input2 = layers.Input(shape = (773,))
    
    features1 = dbn(input1)
    features2 = dbn(input2)
    
    concatenated_features = layers.Concatenate()([features1, features2])
    
    fc1 = layers.Dense(400, activation = 'relu')(concatenated_features)
    fc2 = layers.Dense(200, activation = 'relu')(fc1)
    fc3 = layers.Dense(100, activation = 'relu')(fc2)
    output_layer = layers.Dense(2, activation = 'softmax')(fc3)
    
    siamese_model = tf.keras.Model(inputs=[input1, input2], outputs=output_layer)
    
    return siamese_model

In [ ]:
from sklearn.model_selection import train_test_split

# Convert ZipDataset to numpy arrays
dataset_array = np.array(list(dataset.as_numpy_iterator()))
# Split ratio for train-test split
split_ratio = 0.9

# Split the paired dataset and labels dataset into train and test
train_dataset, test_dataset = train_test_split(
    dataset_array, test_size=1-split_ratio)

# Print the sizes of train and test datasets
print('Train Dataset Size:', len(train_dataset))
print('Test Dataset Size:', len(test_dataset))

In [ ]:
for i in range(len(train_dataset)//2):
    train_dataset[2*i][0][0], train_dataset[2*i][0][1] = train_dataset[2*i][0][1], train_dataset[2*i][0][0]
    train_dataset[2*i][1][0], train_dataset[2*i][1][1] = train_dataset[2*i][1][1], train_dataset[2*i][1][0]


for i in range(len(test_dataset) // 2):
    test_dataset[2*i][0][0], test_dataset[2*i][0][1] = test_dataset[2*i][0][1], test_dataset[2*i][0][0]
    test_dataset[2*i][1][0], test_dataset[2*i][1][1] = test_dataset[2*i][1][1], test_dataset[2*i][1][0]

In [ ]:
# Generate shuffled indices
shuffled_indices = np.random.permutation(len(test_dataset))

# Shuffle train_dataset and train_labels_dataset using the shuffled indices
shuffled_test_dataset = test_dataset[shuffled_indices]


# Print the sizes of shuffled train dataset and labels dataset
print('Shuffled Train Dataset Size:', len(shuffled_test_dataset))


In [ ]:
def scheduler(epoch, lr):
    return lr * 0.99

# Create the learning rate scheduler callback
lr_scheduler = callbacks.LearningRateScheduler(scheduler)

# Set the initial learning rate
initial_learning_rate = 0.01

In [ ]:
#siamese_model = create_siamese_network()

In [ ]:
input1 = []
input2 = []
labels1 = []
labels2 = []
for i in range(len(train_dataset)):
    input1.append(train_dataset[i][0][0])
    input2.append(train_dataset[i][0][1])
    labels1.append(train_dataset[i][1][0][0])
    labels2.append(train_dataset[i][1][1][0])
input1 = np.array(input1)
input2 = np.array(input2)
labels1 = np.array(labels1)
labels2 = np.array(labels2)

In [ ]:
test_input1 = []
test_input2 = []
test_labels1 = []
test_labels2 = []
for i in range(len(shuffled_test_dataset)):
    test_input1.append(shuffled_test_dataset[i][0][0])
    test_input2.append(shuffled_test_dataset[i][0][1])
    test_labels1.append(shuffled_test_dataset[i][1][0][0])
    test_labels2.append(shuffled_test_dataset[i][1][1][0])
test_input1 = np.array(test_input1)
test_input2 = np.array(test_input2)

In [ ]:
test_labels = []
for i in range(len(shuffled_test_dataset)):
    x = [test_labels1[i], test_labels2[i]]
    test_labels.append(x)
test_labels = np.array(test_labels)

In [ ]:
labels = []
for i in range(len(train_dataset)):
    x = [labels1[i], labels2[i]]
    labels.append(x)
labels = np.array(labels)

In [ ]:
labels

In [ ]:
input1.shape

In [ ]:
# Create and compile the Siamese model
siamese_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
siamese_model.fit([input1, input2], labels, epochs=1000, batch_size=1000, callbacks = [lr_scheduler])

In [ ]:

# Evaluate the Siamese model on the test dataset
test_loss, test_accuracy = siamese_model.evaluate([test_input1, test_input2], test_labels)

# Print the accuracy
print('Test Accuracy:', test_accuracy*100)

In [ ]:
def preprocess_for_siamese_input(fen1, fen2):
    bitstring1 = np.array([fen2bitstring(fen1)])
    bitstring2 = np.array([fen2bitstring(fen2)])
    siamese_input = [bitstring1, bitstring2]
    return siamese_input

In [ ]:
fen1 = "3r3k/p4pq1/5Q1p/4pB2/7P/P1N5/1PP2P2/6RK b - - 0 34"  #position with queen blunder
fen2 = "3r3k/p4pQ1/7p/4pB2/7P/2N5/PPP2P2/6RK b - - 0 34"   #best move acc. to stockfish

fin1 = preprocess_for_siamese_input(fen1, fen2)
x = siamese_model.predict(fin1)
x

In [ ]:
fin2 = preprocess_for_siamese_input(fen2, fen1)
y = siamese_model.predict(fin2)
y

In [ ]:
siamese_model.save("siamese_325000.h5")